<a href="https://colab.research.google.com/github/alexandergribenchenko/Data_Science_Toolkit/blob/main/Visualization/NB_02_Pareto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
2+3

5

In [34]:
import pandas as pd
from plotly.graph_objects import Figure, Scatter, Bar

In [41]:

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Creamos un dataframe con dos variables categóricas
data = sns.load_dataset('tips')
data

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [62]:
import pandas as pd

def pareto_distribution_with_window(dataframe, value_col, group_cols, window_cols):
    """
    Genera un DataFrame con la distribución de Pareto, incluyendo cálculos de ventana para subgrupos definidos.

    Args:
        dataframe (pd.DataFrame): El DataFrame de entrada.
        value_col (str): Nombre de la columna para calcular la distribución (por ejemplo, 'tip').
        group_cols (list): Lista de columnas para agrupar los datos (por ejemplo, ['day', 'sex']).
        window_cols (list): Lista de columnas para definir ventanas adicionales (por ejemplo, ['time']).

    Returns:
        pd.DataFrame: DataFrame con los subgrupos, valores totales, porcentaje, porcentaje acumulado y total acumulado.
    """
    # Combinar las columnas para las ventanas
    all_group_cols = window_cols + group_cols

    # Agrupar por las columnas combinadas y calcular la suma
    grouped = dataframe.groupby(all_group_cols, as_index=False, observed=False)[value_col].sum()

    # Renombrar la columna para reflejar que es la suma total
    grouped.rename(columns={value_col: 'total_value'}, inplace=True)

    # Calcular el porcentaje y total acumulado dentro de cada ventana
    grouped['percentage'] = grouped.groupby(window_cols)['total_value'].transform(lambda x: (x / x.sum()) * 100)
    grouped['cumulative_percentage'] = grouped.groupby(window_cols)['percentage'].cumsum()
    grouped['cumulative_total_value'] = grouped.groupby(window_cols)['total_value'].cumsum()

    # Ordenar por ventana y porcentaje descendente
    grouped = grouped.sort_values(by=window_cols + ['percentage'], ascending=[True] * len(window_cols) + [False]).reset_index(drop=True)

    return grouped

In [64]:
import pandas as pd

def pareto_distribution_with_window(dataframe, value_col, group_cols, window_cols):
    """
    Genera un DataFrame con la distribución de Pareto, incluyendo cálculos de ventana para subgrupos definidos.

    Args:
        dataframe (pd.DataFrame): El DataFrame de entrada.
        value_col (str): Nombre de la columna para calcular la distribución (por ejemplo, 'tip').
        group_cols (list): Lista de columnas para agrupar los datos (por ejemplo, ['day', 'sex']).
        window_cols (list): Lista de columnas para definir ventanas adicionales (por ejemplo, ['time']).

    Returns:
        pd.DataFrame: DataFrame con los subgrupos, valores totales, porcentaje, porcentaje acumulado y total acumulado.
    """
    # Combinar las columnas para las ventanas
    all_group_cols = window_cols + group_cols

    # Agrupar por las columnas combinadas y calcular la suma
    grouped = dataframe.groupby(all_group_cols, as_index=False, observed=False)[value_col].sum()

    # Renombrar la columna para reflejar que es la suma total
    grouped.rename(columns={value_col: 'total_value'}, inplace=True)

    # Calcular el porcentaje y total acumulado dentro de cada ventana
    grouped['percentage'] = grouped.groupby(window_cols, observed=False)['total_value'].transform(lambda x: (x / x.sum()) * 100)
    grouped['cumulative_percentage'] = grouped.groupby(window_cols, observed=False)['percentage'].cumsum()
    grouped['cumulative_total_value'] = grouped.groupby(window_cols, observed=False)['total_value'].cumsum()

    # Ordenar por ventana y porcentaje descendente
    grouped = grouped.sort_values(by=window_cols + ['percentage'], ascending=[True] * len(window_cols) + [False]).reset_index(drop=True)

    return grouped


In [80]:
import pandas as pd

def pareto_distribution_with_window(dataframe, value_col, group_cols, window_cols):
    """
    Genera un DataFrame con la distribución de Pareto, incluyendo cálculos de ventana para subgrupos definidos
    y manteniendo el orden de las columnas de ventana mientras ordena internamente las filas por estas columnas.

    Args:
        dataframe (pd.DataFrame): El DataFrame de entrada.
        value_col (str): Nombre de la columna para calcular la distribución (por ejemplo, 'tip').
        group_cols (list): Lista de columnas para agrupar los datos (por ejemplo, ['day', 'sex']).
        window_cols (list): Lista de columnas para definir ventanas adicionales (por ejemplo, ['time']).

    Returns:
        pd.DataFrame: DataFrame con los subgrupos, valores totales, porcentaje, porcentaje acumulado y total acumulado.
    """
    # Ordenar alfabéticamente las filas dentro de cada grupo de las columnas de ventana
    dataframe = dataframe.sort_values(by=window_cols, ascending=[True] * len(window_cols))

    # Asegurarse de que las columnas de ventana estén ordenadas alfabéticamente
    window_cols_sorted = window_cols

    # Combinar las columnas para las ventanas, ahora con las columnas ordenadas
    all_group_cols = window_cols_sorted + group_cols

    # Agrupar por las columnas combinadas y calcular la suma
    grouped = dataframe.groupby(all_group_cols, as_index=False, observed=False)[value_col].sum()

    # Renombrar la columna para reflejar que es la suma total
    grouped.rename(columns={value_col: 'total_value'}, inplace=True)

    # Calcular el porcentaje y total acumulado dentro de cada ventana
    grouped['percentage'] = grouped.groupby(window_cols_sorted, observed=False)['total_value'].transform(lambda x: (x / x.sum()) * 100)
    grouped['cumulative_total_value'] = grouped.groupby(window_cols_sorted, observed=False)['total_value'].cumsum()
    grouped['cumulative_percentage'] = grouped.groupby(window_cols_sorted, observed=False)['percentage'].cumsum()

    # Ordenar por las columnas de ventana en el orden que se proporcionan en la lista y el porcentaje descendente
    grouped = grouped.sort_values(by=window_cols_sorted + ['percentage'], ascending=[False] * len(window_cols_sorted) + [False]).reset_index(drop=True)

    return grouped


In [81]:
pareto_df = pareto_distribution_with_window(data, value_col='tip', group_cols=['day', 'sex'], window_cols=['time', 'smoker'])
pareto_df

,time,smoker,day,sex,total_value,percentage,cumulative_total_value,cumulative_percentage
0,Dinner,No,Sun,Male,133.96,40.416352,284.84,85.937547
1,Dinner,No,Sat,Male,104.21,31.440640,115.46,34.834817
2,Dinner,No,Sun,Female,46.61,14.062453,331.45,100.000000
3,Dinner,No,Sat,Female,35.42,10.686378,150.88,45.521195
4,Dinner,No,Fri,Male,5.00,1.508523,8.00,2.413637
5,Dinner,No,Fri,Female,3.25,0.980540,11.25,3.394177
6,Dinner,No,Thur,Female,3.00,0.905114,3.00,0.905114
7,Dinner,No,Thur,Male,0.00,0.000000,0.00,0.000000
8,Dinner,Yes,Sat,Male,77.74,36.222160,104.77,48.816513
9,Dinner,Yes,Sun,Male,52.82,24.610940,200.62,93.476843


In [82]:
pareto_df.total_value.sum()

731.5799999999999

In [61]:
def pareto_distribution(dataframe, value_col, group_cols):
    """
    Genera un DataFrame con la distribución de Pareto, incluyendo:
    - Total de valores
    - Porcentaje de cada grupo
    - Porcentaje acumulado
    - Total acumulado

    Args:
        dataframe (pd.DataFrame): El DataFrame de entrada.
        value_col (str): Nombre de la columna para calcular la distribución (por ejemplo, 'tip').
        group_cols (list): Lista de columnas para agrupar los datos (por ejemplo, ['day', 'sex']).

    Returns:
        pd.DataFrame: DataFrame con los subgrupos, valores totales, porcentaje, porcentaje acumulado y total acumulado.
    """
    # Agrupar por las columnas especificadas y calcular la suma del valor, con observed=False para evitar el warning
    grouped = dataframe.groupby(group_cols, as_index=False, observed=False)[value_col].sum()

    # Renombrar la columna para reflejar que es la suma total
    grouped.rename(columns={value_col: 'total_value'}, inplace=True)

    # Calcular el porcentaje del total
    total_sum = grouped['total_value'].sum()
    grouped['percentage'] = (grouped['total_value'] / total_sum) * 100

    # Ordenar por porcentaje en orden descendente
    grouped = grouped.sort_values('percentage', ascending=False).reset_index(drop=True)

    # Calcular el porcentaje acumulado
    grouped['cumulative_percentage'] = grouped['percentage'].cumsum()

    # Calcular el total acumulado
    grouped['cumulative_total_value'] = grouped['total_value'].cumsum()

    return grouped

In [60]:
pareto_df = pareto_distribution(data, value_col='tip', group_cols=['day', 'sex'])
pareto_df

,day,sex,total_value,percentage,cumulative_percentage,cumulative_total_value
0,Sun,Male,186.78,25.531042,25.531042,186.78
1,Sat,Male,181.95,24.870828,50.401870,368.73
2,Thur,Male,89.41,12.221493,62.623363,458.14
3,Thur,Female,82.42,11.266027,73.889390,540.56
4,Sat,Female,78.45,10.723366,84.612756,619.01
5,Sun,Female,60.61,8.284808,92.897564,679.62
6,Fri,Male,26.93,3.681074,96.578638,706.55
7,Fri,Female,25.03,3.421362,100.000000,731.58


In [35]:
def graph_pareto(dataframe, value_col, percentage_col, cumulative_col, x_col, extra_cols):
    # Copiar y ordenar los datos
    df = dataframe.copy()
    df = df.reset_index(drop=True)

    # Identificar la primera barra donde el porcentaje acumulado >= 80
    first_above_80 = df[df[cumulative_col] >= 80].index.min()

    # Determinar los colores para las barras basados en el porcentaje acumulado
    bar_colors = [
        'rgb(71, 71, 135)' if idx <= first_above_80 else 'rgb(112, 111, 211)'
        for idx in df.index
    ]

    # Determinar los colores para la línea y los puntos de porcentaje acumulado (rojo oscuro a rojo más claro)
    line_and_marker_colors = [
        'rgb(192, 57, 43)' if idx <= first_above_80 else 'rgb(255, 165, 123)'
        for idx in df.index
    ]

    # Crear etiquetas de texto únicamente con las columnas en extra_cols
    def create_hovertext(row):
        return "<br>".join([f"{col}: {row[col]}" for col in extra_cols])

    # Hovertext para las barras
    bar_hovertext = [create_hovertext(row) for _, row in df.iterrows()]

    # Hovertext para la línea de porcentaje acumulado
    line_hovertext = [create_hovertext(row) for _, row in df.iterrows()]

    # Texto sobre las barras (manteniendo Count y Percentage)
    bar_text = [
        f"{value} ({percentage:.2f}%)"
        for value, percentage in zip(df[value_col], df[percentage_col])
    ]

    data = [
        Bar(
            name="Count",
            x=df[x_col],
            y=df[value_col],
            marker={"color": bar_colors},
            hovertext=bar_hovertext,
            hoverinfo="text",
            text=bar_text,
            textposition="outside",
            textfont={"size": 10},
        ),
        Scatter(
            name="Percentage",
            x=df[x_col],
            y=df[cumulative_col],
            yaxis="y2",
            mode='lines+markers',
            line={
                "color": line_and_marker_colors[0],
                "width": 3
            },
            marker=dict(
                color=line_and_marker_colors,
                size=8,
                line=dict(width=1, color='rgb(0,0,0)')
            ),
            hovertext=line_hovertext,
            hoverinfo="text",
        ),
    ]

    layout = {
        "title": {
            'text': f"Pareto Analysis",
            'font': dict(size=30)
        },
        "font": {
            "size": 14,
            "color": "rgb(44, 44, 84)",
            "family": "Times New Roman, monospace"
        },
        "margin": {
            "b": 50,
            "l": 50,
            "r": 50,
            "t": 50,
        },
        "height": 400,
        "plot_bgcolor": "rgb(255, 255, 255)",
        "legend": {
            "x": 0.79,
            "y": 1.2,
            "font": {
                "size": 12,
                "color": "rgb(44, 44, 84)",
                "family": "Courier New, monospace"
            },
            'orientation': 'h',
        },
        "yaxis": {
            "title": f"Count {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
        },
        "yaxis2": {
            "side": "right",
            "range": [0, 100],
            "title": f"Percentage {value_col}",
            "titlefont": {
                "size": 16,
                "color": "rgb(71, 71, 135)",
                "family": "Courier New, monospace"
            },
            "overlaying": "y",
            "ticksuffix": " %",
        },
    }

    # Crear el gráfico
    fig = Figure(data=data, layout=layout)
    fig.show()

In [36]:
data = {
    "xml_parent_city_name": ["New York"] * 11,
    "xml_city_name": [
        "New York", "Brooklyn", "Hoboken", "Manhattan", "Union City",
        "Jersey City", "Queens", "West New York", "Newark", "Elizabeth", "Paterson"
    ],
    "distance_from_parent": [0.00, 3.31, 6.44, 6.52, 7.55, 7.65, 7.68, 7.70, 13.13, 14.08, 19.64],
    "leads": [1087, 89, 25, 73, 15, 61, 135, 23, 6, 11, 10],
    "percentage": [70.81, 5.80, 1.63, 4.76, 0.98, 3.97, 8.79, 1.50, 0.39, 0.72, 0.65],
    "cumulative_percentage": [70.81, 76.61, 78.24, 83.00, 83.98, 87.95, 96.74, 98.24, 98.63, 99.35, 100.00],
}

df = pd.DataFrame(data)

df_plot = df
df_plot

,xml_parent_city_name,xml_city_name,distance_from_parent,leads,percentage,cumulative_percentage
0,New York,New York,0.00,1087,70.81,70.81
1,New York,Brooklyn,3.31,89,5.80,76.61
2,New York,Hoboken,6.44,25,1.63,78.24
3,New York,Manhattan,6.52,73,4.76,83.00
4,New York,Union City,7.55,15,0.98,83.98
5,New York,Jersey City,7.65,61,3.97,87.95
6,New York,Queens,7.68,135,8.79,96.74
7,New York,West New York,7.70,23,1.50,98.24
8,New York,Newark,13.13,6,0.39,98.63
9,New York,Elizabeth,14.08,11,0.72,99.35


In [39]:
graph_pareto(
    dataframe=df_plot,
    value_col="leads",
    percentage_col="percentage",
    cumulative_col="cumulative_percentage",
    x_col="xml_city_name",
  extra_cols=list(df_plot.columns)
)